# ARNAB CHATTERJEE
# Assignment 3

In [70]:
pip install vecstack

Note: you may need to restart the kernel to use updated packages.


In [72]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from collections import Counter #for Smote,

import warnings
warnings.filterwarnings("ignore")

In [74]:
import pandas as pd
from collections import Counter

data = pd.read_csv('RevisedHomesiteTrain1.csv')

# defining features and target variable
X = data.drop(columns=['QuoteConversion_Flag'])
y = data['QuoteConversion_Flag']

# Checking the original dataset class distribution
print("Original dataset shape:", Counter(y))

Original dataset shape: Counter({0: 52738, 1: 12262})


In [44]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from collections import Counter

# Assuming X, y are your features and target respectively

# Applying SMOTE to balance the dataset
print("Original dataset shape:", Counter(y))

smote = SMOTE(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Resampled dataset shape:", Counter(y_resampled))

# Spliting the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)



Original dataset shape: Counter({0: 52738, 1: 12262})
Resampled dataset shape: Counter({0: 52738, 1: 26369})


In [50]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Step 1: Loading the dataset
train_data = pd.read_csv('RevisedHomesiteTrain1.csv')

# Check if the dataset was loaded correctly or not
print("Dataset loaded successfully!")
print(train_data.head())

# Specifying the target column (adjust this if necessary)
target_column = 'QuoteConversion_Flag'

# Step 2: Separating features and target variable
X = train_data.drop(columns=[target_column])
y = train_data[target_column]

# Step 3: Spliting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Applying SMOTE to balance the classes
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Printing class distribution after SMOTE
print("Class distribution after applying SMOTE:")
print(y_train_smote.value_counts())

# Step 5: Defining models
models = {
    'MLP': MLPClassifier(max_iter=100, random_state=42),
    'SVM': SVC(probability=True, kernel='linear', max_iter=1000, random_state=42),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    'SGDClassifier': SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, random_state=42)
}

# Step 6: Training and evaluate each model
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train_smote, y_train_smote)

    # Predicting and evaluate=ing
    y_pred = model.predict(X_val)
    print(f"Model: {model_name}")
    print(classification_report(y_val, y_pred))

    # Calculating AUC if the model supports predict_proba
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_val)[:, 1]
        print("AUC:", roc_auc_score(y_val, y_pred_proba))

    print("="*50)

Dataset loaded successfully!
   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                 2                 1               17               23   
1                 5                 9                6                8   
2                 4                 6                7               12   
3                15                23                3                2   
4                 4                 6                8               13   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0               17               23               15               22   
1                6                8                5                7   
2                7               12                6               10   
3                3                2                2                2   
4                8               13                7               11   

   CoverageField4A  CoverageField4B  ...  PropertyField38_N  \
0               16

In [52]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Assuming X_train_smote_scaled and y_train_smote are already defined
# Sample your data
X_train_sample = pd.DataFrame(X_train_smote_scaled).sample(frac=0.5, random_state=42)
y_train_sample = y_train_smote.loc[X_train_sample.index]

# Ensure the data arrays are writeable
X_train_sample = X_train_sample.copy()
y_train_sample = y_train_sample.copy()

# Check and print the writeable flag (optional)
# print("X_train_sample writeable:", X_train_sample.values.flags.writeable)
# print("y_train_sample writeable:", y_train_sample.values.flags.writeable)

# Scale the features to improve convergence
scaler = StandardScaler()
X_train_sample_scaled = scaler.fit_transform(X_train_sample)

# Define base estimators
estimators = [
    ('lr', LogisticRegression(max_iter=1000)),
    ('svc', SVC(probability=True, max_iter=1000)),
    ('mlp', MLPClassifier(max_iter=1000))
]

# Define the stacking classifier
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    n_jobs=1  # Use a single core to avoid parallel processing issues
)

# Define parameter distributions for random search
param_dist = {
    'final_estimator__C': np.logspace(-4, 4, 10)
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=stacking_clf,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    n_jobs=1  # Use a single core to avoid parallel processing issues
)

# Fit the model
random_search.fit(X_train_sample_scaled, y_train_sample)

# Print the best parameters found
print("Best parameters found:", random_search.best_params_)

import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

Best parameters found: {'final_estimator__C': 0.0001}


In [54]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import warnings
from sklearn.exceptions import ConvergenceWarning
import numpy as np

warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Step 1: Loading the dataset (sample 1000 rows for fast execution)
train_data = pd.read_csv('RevisedHomesiteTrain1.csv').sample(n=1000, random_state=42)
target_column = 'QuoteConversion_Flag'

# Step 2: Separating features and target variable
X = train_data.drop(columns=[target_column])
y = train_data[target_column]

# Step 3: Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Applying SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Step 5: Scaling features
scaler = StandardScaler()
X_train_smote_scaled = scaler.fit_transform(X_train_smote)
X_val_scaled = scaler.transform(X_val)

# Step 6: Defining base models
base_models = [
    ('rf', RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)),
    ('svm', SVC(kernel='linear', probability=True, random_state=42, max_iter=2000)),  # Increased max_iter
    ('mlp', MLPClassifier(max_iter=200, random_state=42, batch_size=64)),  # Modified parameters for MLP
    ('sgd', SGDClassifier(loss='log_loss', max_iter=500, random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]

# Step 7: Defining stacking model
final_estimator = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
stacking_model = StackingClassifier(estimators=base_models, final_estimator=final_estimator, n_jobs=1)  # Set n_jobs to 1

# Step 8: Defining hyperparameter grid
param_grid = {
    'final_estimator__n_estimators': [50, 100],
    'final_estimator__max_depth': [None, 10],
    'final_estimator__min_samples_split': [2, 5],
    'final_estimator__min_samples_leaf': [1, 2],
    'final_estimator__bootstrap': [True, False]
}

# Step 9: Hyperparameter tuning with RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=stacking_model,
    param_distributions=param_grid,
    n_iter=5,  # Limiting the combinations for faster execution
    cv=3,
    scoring='accuracy',
    n_jobs=1,  # Set to 1 to avoid parallel errors
    random_state=42,
    verbose=1
)

# Training with a subset of data for faster execution
X_train_sample = pd.DataFrame(X_train_smote_scaled).sample(frac=0.5, random_state=42).to_numpy()
X_train_sample = np.array(X_train_sample, copy=True, order='C')  # Convert to writable array
y_train_sample = y_train_smote.loc[X_train_sample[:, 0].argsort()[:len(X_train_sample)]]

random_search.fit(X_train_sample, y_train_sample)

# Step 10: Evaluating all models
print("Evaluating Base Models:")
for name, model in base_models:
    model.fit(X_train_smote_scaled, y_train_smote)  # Train base models
    y_val_pred_base = model.predict(X_val_scaled)
    accuracy = accuracy_score(y_val, y_val_pred_base)
    print(f"Model: {name}, Accuracy: {accuracy:.4f}")

# Step 11: Evaluating best model from hyperparameter tuning
print("\nEvaluating Stacking Model (Best Model):")
best_model = random_search.best_estimator_
y_val_pred_best = best_model.predict(X_val_scaled)
best_model_accuracy = accuracy_score(y_val, y_val_pred_best)
print(f"Best Model Accuracy: {best_model_accuracy:.4f}")

# Calculating AUC for best model
if hasattr(best_model, 'predict_proba'):
    y_val_proba_best = best_model.predict_proba(X_val_scaled)[:, 1]
    best_model_auc = roc_auc_score(y_val, y_val_proba_best)
    print(f"Best Model AUC: {best_model_auc:.4f}")

# Step 12: Final classification report
print("\nClassification Report for Best Model:")
print(classification_report(y_val, y_val_pred_best))

print("\nBest Hyperparameters:", random_search.best_params_)


Fitting 3 folds for each of 5 candidates, totalling 15 fits
Evaluating Base Models:
Model: rf, Accuracy: 0.8350
Model: svm, Accuracy: 0.8400
Model: mlp, Accuracy: 0.8700
Model: sgd, Accuracy: 0.8650
Model: dt, Accuracy: 0.8150
Model: knn, Accuracy: 0.5500

Evaluating Stacking Model (Best Model):
Best Model Accuracy: 0.8250
Best Model AUC: 0.3984

Classification Report for Best Model:
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       165
           1       0.00      0.00      0.00        35

    accuracy                           0.82       200
   macro avg       0.41      0.50      0.45       200
weighted avg       0.68      0.82      0.75       200


Best Hyperparameters: {'final_estimator__n_estimators': 100, 'final_estimator__min_samples_split': 5, 'final_estimator__min_samples_leaf': 2, 'final_estimator__max_depth': 10, 'final_estimator__bootstrap': True}


In [60]:
# Step 13: Saving model predictions
# Ensuring the test dataset has the same columns as the training data
test_data = pd.read_csv('RevisedHomesiteTest1.csv')  # Assuming test data file name
test_data_aligned = test_data[X.columns]

# Checking if any columns are missing after alignment
missing_cols = set(X.columns) - set(test_data_aligned.columns)
if missing_cols:
    print(f"Warning: The following columns are missing in the test data: {missing_cols}")

# Making predictions using the aligned test dataset
test_predictions = best_model.predict_proba(test_data_aligned)[:, 1]

# Creating a submission DataFrame
submission_df = pd.DataFrame({
    'QuoteNumber': test_data['QuoteNumber'],
    'QuoteConversion_Flag': test_predictions
})

# Saving the submission to CSV
submission_file = 'result.csv'
submission_df.to_csv(submission_file, index=False)
print(f"Submission file saved as '{submission_file}'")


Submission file saved as 'result.csv'
